<b><font size=5> Backtest Misc

Examples of backtesting for 
* multiple series 
* multiple models

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

from orbit.lgt import LGT
from orbit.dlt import DLT
from orbit.backtest.backtest import TimeSeriesSplitter, Backtest
from orbit.utils.metrics import mape, smape, wmape
from orbit.backtest.functions import run_multi_series_backtest

## Load data

In [2]:
data_path = "../examples/data/m4_weekly.csv"
data = pd.read_csv(data_path, parse_dates=['date'])

In [3]:
print("Total keys: {}".format(len(data['key'].unique())))
data.head(3)

Total keys: 3


,key,week_num,value,date
0,W1,2,1089.20,1970-01-05
1,W1,3,1078.91,1970-01-12
2,W1,4,1079.88,1970-01-19


## Settings

In [4]:
# data settings
date_col = "date"
response_col = "value"
key_col = "key"
seasonality = 52

# backtest settings
min_train_len = 120
forecast_len = 13
incremental_len = 5
n_splits = 1
window_type = "expanding"
seed = 2019

In [5]:
lgt_map = LGT(
    response_col=response_col,
    date_col=date_col,
    seasonality=seasonality,
    seed=seed,
    infer_method='map',
    predict_method='map',
    is_multiplicative=True
)

dlt_map = DLT(
    response_col=response_col,
    date_col=date_col,
    seasonality=seasonality,
    seed=seed,
    infer_method='map',
    predict_method='map',
    global_trend_option='loglinear',
    is_multiplicative=True
)


In [6]:
bt_models = {
    'LGT-MAP': lgt_map,
    'DLT-MAP': dlt_map
}

## Backtest Multiple Series

In [7]:
bt_result, bt_scores = run_multi_series_backtest(
    data=data, 
    response_col=response_col, 
    key_col=key_col, 
    date_col=date_col,
    model=lgt_map,
    min_train_len=min_train_len, 
    incremental_len=incremental_len, 
    forecast_len=forecast_len, 
    predicted_col='prediction', 
    n_splits=n_splits,
    window_type = window_type
)

100%|██████████| 3/3 [00:09<00:00,  3.11s/it]


In [8]:
bt_scores.head()

,wmape,smape,n_splits,key
0,0.021875,0.021738,1,W1
1,0.016515,0.016587,1,W2
2,0.007889,0.007881,1,W3


## Backtest Multiple Models (and Multiple Series)

In [9]:
bt_result_list = []
bt_scores_list = []
for mod_name, mod in bt_models.items():
    bt_result, bt_scores = run_multi_series_backtest(
        data=data, 
        response_col=response_col, 
        key_col=key_col, 
        date_col=date_col,
        model=lgt_map,
        min_train_len=min_train_len, 
        incremental_len=incremental_len, 
        forecast_len=forecast_len, 
        predicted_col='prediction', 
        n_splits=n_splits,
        window_type = window_type
    )
    bt_result['model'] = mod_name
    bt_scores['model'] = mod_name
    bt_result_list.append(bt_result)
    bt_scores_list.append(bt_scores)

100%|██████████| 3/3 [00:08<00:00,  2.72s/it]


In [10]:
bt_result = pd.concat(bt_result_list, axis=0)
bt_scores = pd.concat(bt_scores_list, axis=0)

In [11]:
bt_scores.head()

,wmape,smape,n_splits,key,model
0,0.021875,0.021738,1,W1,LGT-MAP
1,0.016515,0.016587,1,W2,LGT-MAP
2,0.007889,0.007881,1,W3,LGT-MAP
0,0.021875,0.021738,1,W1,DLT-MAP
1,0.016515,0.016587,1,W2,DLT-MAP
